# Handling Errors and Status Codes

- HTTP status codes communicate the outcome of an API request, and handling them correctly is key to robust automation.
- A simple `200 OK` means success, while codes like `404 Not Found` or `500 Internal Server Error` indicate different failure modes.
- Check status code and `response.ok` to raise errors automatically, and inspect error details for troubleshooting.

## Understanding HTTP Status Codes

Status codes are grouped by their first digit.
- **1XX (informational)**

- **2XX (success)**:
    - `200 OK`
    - `201 Created`,
    - `202 Accepted`
    - `204 No Content`
    - `206 Partial Content`
    <br>
- **3XX (redirection)**:
    - `300 Multiple Choices`
    - `301 Moved Permanently`
    - `302 Found`
    <br>
- **4XX (client error)**:
    - `400 Bad Request`
    - `401 Not Authenticated`
    - `403 Not Authorised`
    - `404 Not Found`
    - `409 Conflict`
    - `422 Invalid Input`
    <br>
- **5XX (server error)**:
    - `500 Internal Server Error`
    - `501 Not Implemented`
    - `502 Bad Gateway`
    - `503 Service Unavailable`
    - `504 Gatewat Timeout`

## Checking `response.status_code`

- After a `requests` call, the integer `response.status_code` tells you the exact HTTP code returned.
- You can compare it directly (e.g., `if resp.status_code == 404:`) to implement custom logic based on the code.
- This explicit check is useful when you need fine-grained control over specific status codes.

In [1]:
GITHUB_ENDPOINT = "https://api.github.com"
HTTPBIN_ENDPOINT = "https://httpbin.org"

In [27]:
import requests

urls = {
    "ok": f"{GITHUB_ENDPOINT}/zen",
    "not_found": f"{GITHUB_ENDPOINT}/nonexistentendpoint"
}

for description, url in urls.items():
    response = requests.get(url, timeout=5)
    print(f"{description}: status {response.status_code}")

ok: status 200
not_found: status 404


## Using `response.ok`

- The boolean `response.ok` is `True` for any status code below 400 (1xx, 2xx, 3xx) and `False` for 4xx/5xx errors.
- This provides a quick success/failure check without examining the numeric code directly.
- It’s a handy shorthand when you only need to know if the request broadly succeeded.

In [28]:
import requests

urls = {
    "ok": f"{GITHUB_ENDPOINT}/zen",
    "not_found": f"{GITHUB_ENDPOINT}/nonexistentendpoint"
}

for description, url in urls.items():
    response = requests.get(url, timeout=5)
    print(f"{description}: ok? {"Yes" if response.ok else f"No. Failed with status {response.status_code}"}")

ok: ok? Yes
not_found: ok? No. Failed with status 404


## Automatic Error Raising with `raise_for_status()`

- Calling `response.raise_for_status()` will do nothing on **1XX**, **2XX** and **3XX** codes but raise an `HTTPError` on **4XX/5XX**.
- This follows the EAFP (“Easier to Ask Forgiveness than Permission”) style: try the request, and catch errors if they occur.
- The caught exception carries the original `response` in its `response` attribute, letting you inspect headers and body.

In [9]:
import requests
import json

urls = {
    "ok": f"{GITHUB_ENDPOINT}/zen",
    "not_found": f"{GITHUB_ENDPOINT}/nonexistentendpoint"
}

for url in urls.values():
    print(f"Requesting: {url}")
    try:
        res = requests.get(url, timeout=5)
        res.raise_for_status() # raise ERROR if 4XX/5XX
        print("\tSuccess!")
        print(f"\t{res.text}")
    except requests.exceptions.HTTPError as err:
        print(f"\tHTTPError: {err} (status {err.response.status_code})")
        try:
            details = err.response.json()
            print("\tError details:")
            print(json.dumps(details, indent=2))
        except ValueError:
            print(f"\tNon-JSON response body: {err.response.text[:100]}")

Requesting: https://api.github.com/zen
	Success!
	Practicality beats purity.
Requesting: https://api.github.com/nonexistentendpoint
	HTTPError: 404 Client Error: Not Found for url: https://api.github.com/nonexistentendpoint (status 404)
	Error details:
{
  "message": "Not Found",
  "documentation_url": "https://docs.github.com/rest",
  "status": "404"
}


## Common Pitfalls & How to Avoid Them

- **Not checking errors:** Treating any response as success can mask failures. Always use `ok` or `raise_for_status()`.
- **Catching too broadly:** A generic `except Exception:` hides HTTP errors. Catch `HTTPError` specifically.
- **Ignoring error bodies:** APIs often return JSON error messages; inspect `response.text` or `response.json()`.